<a href="https://colab.research.google.com/github/manii5228/fish_classification/blob/main/fishclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-- step1: Installing the packages--
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import zipfile
import os

print("Tensorflow version:",tf.__version__)
print("all packages installed")

Tensorflow version: 2.19.0
all packages installed


In [ ]:
# step2: uploading the the datasets

from google.colab import files


# : Upload the ZIP file
uploaded = files.upload()  # Pick your dataset.zip file

#  Get the filename
zip_filename = next(iter(uploaded))

# Extract the ZIP
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(".")

print(f"✅ Extracted: {zip_filename}")

# Optional - Check the folder structure
os.listdir(".")


In [ ]:
import tensorflow as tf

IMG_SIZE = (224, 224)
BATCH_SIZE = 12
AUTOTUNE = tf.data.AUTOTUNE

# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "NA_Fish_Dataset/train",
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "NA_Fish_Dataset/val",
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

print("Train classes:", train_ds.class_names)
print("Val classes:", val_ds.class_names)

class_names = train_ds.class_names
num_classes = len(class_names)

'''# ✅ Force all images to RGB
def force_rgb(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.image.grayscale_to_rgb(image) if tf.shape(image)[-1] == 1 else image

    image.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3]) # Ensure 3 channels
    return image, label

# ✅ Apply the RGB fix, shuffle, and prefetch for both train and validation folders
train_ds = train_ds.map(force_rgb, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.map(force_rgb, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)'''
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
#Data Augmentation Layer

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
])
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

#Load MobileNetV2 Base Model

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)


base_model.trainable = False  # Freeze pre-trained layers

#Build the Final Model

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

#Compile and Train
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,

    validation_data=val_ds,
    epochs=25
)


# Unfreeze top layers only (e.g., last 50 layers)
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train_ds, validation_data=val_ds, epochs=5)



In [ ]:
'''drawing a confusion matrix this helps us to understand at what level our
model is identifing the images from the given classes'''

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=class_names)
disp.plot(xticks_rotation=45)
plt.show()


In [ ]:
'''this code helps in making the above confusion matrix into text
for easier debugging'''
from sklearn.metrics import confusion_matrix
import numpy as np

# Assume y_true and y_pred are already prepared
cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))

print("\nConfusion Matrix (Text Format):\n")
print("Predicted →", "\t".join(class_names))
for i, row in enumerate(cm):
    row_str = "\t".join(str(x) for x in row)
    print(f"{class_names[i]:<12} → {row_str}")


In [ ]:
'''this code predictis images by picking at random from the val folder'''

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

val_images = val_ds.unbatch().shuffle(100)
for images, label in val_images.take(5):
  img_np=images.numpy()

  input_img = tf.expand_dims(img_np, axis=0)  # Already in right format
  input_img = input_img * 255.0  # Convert to raw range
  input_img = preprocess_input(input_img)
  pred = model.predict(input_img)
  print("Softmax vector:", pred[0])

  pred = model.predict(input_img)
  pred_label = class_names[np.argmax(pred)]
  confidence = np.max(pred)


  plt.imshow(img_np / 255.0)
  plt.title(f"True: {class_names[label.numpy()]}, Pred: {pred_label} ({confidence:.2f})")
  plt.axis('off')
  plt.show()



In [ ]:
#drawing the graph for the train and val loss and accuracy

plt.figure(figsize=(12, 4))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.show()


In [ ]:
##saving the model and downloading it

model.save("fc_model.keras")
print("saved the model fc_model.keras")

# Download model
files.download("fc_model.keras")
model.summary()

In [ ]:
#prediction of the image

from google.colab import files
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.efficientnet import preprocess_input

# Load model
model = load_model("fc_model.keras")

# Upload image
uploaded = files.upload()
image_path = next(iter(uploaded))  # Get uploaded file name

# Load and preprocess the image
img = Image.open(image_path).convert("RGB")
target_size = model.input_shape[1:3]  # e.g., (224, 224)
img = img.resize(target_size)

img_array = np.array(img)              # shape (224, 224, 3), raw RGB
img_array = preprocess_input(img_array)  # scales to [-1, 1]
img_array = np.expand_dims(img_array, axis=0)  # shape (1, 224, 224, 3)

# Predict
predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]
confidence = np.max(predictions)
print("Prediction scores:", predictions)

# Display result
plt.imshow(img)
plt.axis('off')
plt.title(f"Predicted: {predicted_class} ({confidence:.2f})")
plt.show()

print(f"✅ Predicted Class: {predicted_class} | Confidence: {confidence:.2f}")

